In [1]:
%load_ext autoreload
%autoreload 2

In [67]:
import pandas as pd
import numpy as np
import pickle
import sys
import time
import glob
import re
from tqdm.notebook import trange, tqdm
import classifier_utils as cu
import utils
import plotly.figure_factory as ff
from collections import defaultdict
import networkx as nx
from plotly.offline import init_notebook_mode
import plotly.offline as py
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [3]:
with open('user_scores_blobkbot_control_model_hate_abuse.p', 'rb') as f:
    (control_user_scores, blockbot_user_scores) = pickle.load(f)b
    

In [22]:
with open('model_hate_abuse_dataset.p', 'rb') as f:
    model, classes = pickle.load(f)

In [11]:
harassing_users = [str(u) for u, v in blockbot_user_scores.items() if v[0] > 0.5]

In [70]:
with open('../data/senete_harrassing_users.p', 'rb') as f:
    (senete_harrassing_users, senete_harrassing_users_list) = pickle.load(f)

In [72]:
user_tweets = utils.load_user_tweets_by_name('../data/harassers_from_targetted', senete_harrassing_users_list, text_only=False)

In [18]:
user_tweets = utils.load_user_tweets_by_name('../data/BBlevel1_user_tweets', harassing_users, text_only=False)

In [73]:
G = nx.DiGraph()
for _, tweets in user_tweets.items():
    tweets = [t for t in tweets if t.in_reply_to_status_id]
    texts = [t.text for t in tweets]
    p = model.predict_proba(texts)[0][:, 1]
    tweets = np.array(tweets)
    tweets = tweets[p > 0.7]
    
    for t in tweets:
        if G.has_edge(t.user.screen_name, t.in_reply_to_screen_name):
            G[t.user.screen_name][t.in_reply_to_screen_name]['weight'] += 1
            G[t.user.screen_name][t.in_reply_to_screen_name]['tweets'].append(t.text)
        else:
            G.add_edge(t.user.screen_name, t.in_reply_to_screen_name, weight=1)
            G[t.user.screen_name][t.in_reply_to_screen_name]['tweets'] = [t.text]

/usr/lib/python3.6/re.py:212: FutureWarning:

split() requires a non-empty pattern match.



In [77]:
senate_target_users = [
    'SenHydeSmith',
    'SenTinaSmith',
    'gillibrandny',
    'SenJackyRosen',
    'SenCortezMasto',
    'SenKamalaHarris',
    'SenMcSallyAZ',
    'SenJoniErnst',
    'SenatorSinema',
    'SenatorBaldwin',
    'SenatorFischer',
    'SenDuckworth',
    'SenWarren',
    'SenatorHassan',
    'SenFeinstein',
    'PattyMurray',
    'MarshaBlackburn',
    'SenCapito',
    'SenatorCantwell',
    'SenatorShaheen',
    'maziehirono',
    'SenStabenow',
    'SenAmyKlobuchar',
    'SenatorCollins',
    'lisamurkowski'
]

In [87]:
for n in G.nodes:
    if n in senete_harrassing_users_list:
        G.nodes[n]['type'] = 'harasser'
    if n in senate_target_users:
        G.nodes[n]['type'] = 'victim'

In [89]:
nx.write_gml(G, "senete.gml")